# Installations

In [1]:
! pip install faiss-cpu pdfplumber python-docx groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 130.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 106.6 MB/s eta 0:00:00


In [2]:
!pip install langchain

In [3]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.4/483.4 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 46.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

# Imports

In [4]:
import os
import faiss
import json
import numpy as np
from typing import List, Dict
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pdfplumber
import docx
from email import policy
from email.parser import BytesParser
from groq import Groq
import re
from langchain_core.runnables import RunnableLambda
from langchain_core.tools import Tool
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.prompts import PromptTemplate

# Main Code

In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer

# 1. Document Loading & Chunking (Enhanced)

def extract_text_from_pdf(path: str) -> List[Dict]:
    """Extracts text from each page of a PDF."""
    chunks = []
    with pdfplumber.open(path) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            if text:
                chunks.append({"text": text.strip(), "page": i + 1})
    return chunks

def extract_text_from_docx(path: str) -> List[Dict]:
    """Extracts text from a DOCX document."""
    doc = docx.Document(path)
    full_text = "\n".join([para.text for para in doc.paragraphs if para.text.strip()])
    return [{"text": full_text, "page": 1}]

def extract_text_from_email(path: str) -> List[Dict]:
    """Extracts the body from an email file."""
    with open(path, 'rb') as f:
        msg = BytesParser(policy=policy.default).parse(f)
    body = msg.get_body(preferencelist=('plain')).get_content()
    return [{"text": body.strip(), "page": 1}]

def load_and_chunk_document(path: str, chunk_size=1000, chunk_overlap=200) -> List[Dict]:
    """
    Loads a document from the given path, extracts the text, and then chunks it
    using a RecursiveCharacterTextSplitter.
    """
    ext = os.path.splitext(path)[1].lower()
    if ext == ".pdf":
        raw_pages = extract_text_from_pdf(path)
    elif ext == ".docx":
        raw_pages = extract_text_from_docx(path)
    elif ext in [".eml", ".email"]:
        raw_pages = extract_text_from_email(path)
    else:
        raise ValueError("Unsupported file format")

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )

    all_chunks = []
    for page_info in raw_pages:
        page_text = page_info["text"]
        page_number = page_info["page"]

        # Use RecursiveCharacterTextSplitter for chunking
        recursive_chunks = text_splitter.split_text(page_text)

        for chunk_content in recursive_chunks:
            all_chunks.append({
                "content": chunk_content,
                "page": page_number,
                "source_file": os.path.basename(path)
            })

    return all_chunks

from functools import lru_cache

@lru_cache(maxsize=1)
def get_index_and_chunks(doc_path):
    chunks = load_and_chunk_document(doc_path)
    index, embeddings, model = build_faiss_index(chunks)
    return chunks, index, model


In [6]:
# 2. Embedding + FAISS Indexing

def build_faiss_index(chunks: List[Dict], model_name='all-MiniLM-L6-v2'):
    model = SentenceTransformer(model_name)
    texts = [c['content'] for c in chunks]
    embeddings = model.encode(texts)

    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(np.array(embeddings).astype('float32'))

    return index, embeddings, model

In [7]:
# 3. Hybrid Semantic Retrieval

def search_top_chunks(query: str, chunks: List[Dict], model, index, k=5) -> List[Dict]:
    query_vec = model.encode([query])
    D, I = index.search(np.array(query_vec).astype('float32'), k)
    return [chunks[i] for i in I[0]]

In [8]:
def parse_query_with_llm(query: str, groq_client: Groq) -> Dict:
    """
    Uses the LLM to parse a natural language query into a structured JSON object.
    """
    prompt = f"""
    You are an expert at parsing user queries related to insurance policies.
    Your task is to extract key entities from the user's question and structure them into a JSON object.

    The entities to extract are:
    - "age": The age of the person in the query (as an integer).
    - "gender": The gender, if mentioned ("M" or "F").
    - "procedure": The medical procedure or treatment mentioned.
    - "location": The city or location mentioned.
    - "policy_duration": Any mention of how long the policy has been active.
    - "condition": Any other medical condition or context.

    User Query: "{query}"

    Return a clean JSON object. If a value is not found, set it to null.
    """

    try:
        chat_completion = groq_client.chat.completions.create(
            messages=[{"role": "user", "content": prompt}],
            model="llama3-8b-8192",
            temperature=0,
            response_format={"type": "json_object"} # Enforce JSON output
        )
        result = chat_completion.choices[0].message.content
        return json.loads(result)
    except Exception as e:
        print(f"Error parsing query with LLM: {e}")
        return {} # Return an empty dict on failure

In [9]:
#Reranking the chunks for better retrival
from sentence_transformers.cross_encoder import CrossEncoder

def rerank_chunks(query: str, chunks: List[Dict], top_n=3) -> List[Dict]:
    """
    Re-ranks a list of chunks based on their relevance to the query using a CrossEncoder model.
    """
    # Initialize the CrossEncoder model. This model is more powerful but slower than bi-encoders.
    cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

    # Create pairs of [query, chunk_content] for scoring.
    pairs = [[query, chunk['content']] for chunk in chunks]

    # Predict the relevance scores.
    scores = cross_encoder.predict(pairs)

    # Combine chunks with their scores and sort them.
    scored_chunks = list(zip(scores, chunks))
    scored_chunks.sort(reverse=True, key=lambda x: x[0])

    # Return the top N most relevant chunks.
    return [chunk for score, chunk in scored_chunks[:top_n]]

In [10]:
# 5. Prompt Template

prompt_template = PromptTemplate(
    input_variables=["query", "context"],
    template="""
You are an intelligent insurance assistant.

User has asked:
{query}

Refer to the following policy clauses to answer:
---
{context}
---

Important instructions:
- Do NOT assume any medical history or user behavior unless explicitly mentioned in the query.
- Do NOT confuse unrelated medical domains.
- Only use clauses that match the medical situation in the user’s query.
- If a clause is not relevant, do not use it.

Your task:
- Return a decision: Approved / Rejected
- Estimated payout amount (if applicable)
- Justify based on which clause(s)
- Format the result in strict JSON with keys: "decision", "amount", "justification", "clause_reference"
"""
)


In [23]:
# 6. Call the LLM (Groq)

def run_inference(prompt: str, groq_client: Groq) -> Dict:
    try:
        chat_completion = groq_client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": prompt,
                }
            ],
            model="llama3-8b-8192",
        )
        result = chat_completion.choices[0].message.content.strip()
        json_match = result[result.find('{'): result.rfind('}')+1]
        return json.loads(json_match)
    except json.JSONDecodeError as e:
        print(f"JSON parsing error: {e}")
        return {"error": "Could not parse output", "raw": result}
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return {"error": "An unexpected error occurred", "raw": str(e)}

In [12]:
# 7. Reflection Tool to Detect Assumptions

def detect_assumption(prompt: str) -> str:
    return f"Does this reasoning make assumptions not stated in the query?\n{prompt}\n\nBe critical and point them out explicitly."

def check_assumptions_wrapper(p):
    return run_inference(detect_assumption(p), groq_client)

reflection_tool = Tool(
    name="AssumptionChecker",
    func=check_assumptions_wrapper,
    description="Checks if the given reasoning makes unstated assumptions."
)

In [13]:
# 8. Agent Execution

def build_agent(groq_client: Groq):
    tools = [
        Tool(
            name="RetrieveRelevantChunks",
            func=lambda input: search_top_chunks(input['query'], input['chunks'], input['model'], input['index']),
            description="Retrieves relevant document clauses for the query"
        ),
        reflection_tool
    ]

    agent = create_react_agent(
        tools=tools,
        llm=lambda input: run_inference(prompt_template.format(query=input['query'], context="\n\n".join([f"[Clause - Pg {c['page']}] {c['content']}" for c in input['chunks']])), groq_client)
    )
    return AgentExecutor(agent=agent, verbose=True)

In [21]:
# 9. End-to-End Runner

# 9. End-to-End Runner (Enhanced)

def process_query(query: str, doc_path: str, groq_client: Groq):
    parsed_query_data = parse_query_with_llm(query, groq_client)
    print("✅ Parsed Query:", json.dumps(parsed_query_data, indent=2))

    chunks, index, model = get_index_and_chunks(doc_path)

    initial_chunks = search_top_chunks(query, chunks, model, index, k=10)
    reranked_chunks = rerank_chunks(query, initial_chunks, top_n=3)

    context = "\n\n".join([f"[Clause - Pg {c['page']}] {c['content']}" for c in reranked_chunks])
    prompt = prompt_template.format(query=query, context=context)
    return run_inference(prompt, groq_client)

In [15]:
groq_client = Groq(api_key="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")

In [16]:
def ask_a_question(query, doc_path):
    result = process_query(query, doc_path, groq_client)
    print(json.dumps(result, indent=2))


# Testing

In [24]:
queries = ["21M, knee surgery, Pune, 3-month policy", "Can I get coverage for mental illness treatment if admitted to a psychiatric hospital?"]
for query in queries:
  ask_a_question(query, "/content/BajajPolicies.pdf")

✅ Parsed Query: {
  "age": 21,
  "gender": "M",
  "procedure": "knee surgery",
  "location": "Pune",
  "policy_duration": 3,
  "condition": null
}
{
  "decision": "Approved",
  "amount": "~Rs. 3,00,000 to Rs. 5,00,000 depending on the actual medical expenditure",
  "justification": "The knee surgery procedure is included in the list of Day Care Procedures (Annexure I, Clause - Pg 43). The policyholder can claim the medical expenses as per the claims procedure outlined in Clause - Pg 45 and Clause - Pg 46. The estimated payout amount is based on the actual medical expenditure, but is expected to be within the range of Rs. 3,00,000 to Rs. 5,00,000.",
  "clause_reference": "Clause - Pg 43, Clause - Pg 45, Clause - Pg 46"
}
✅ Parsed Query: {
  "age": null,
  "gender": null,
  "procedure": "mental illness treatment",
  "location": "psychiatric hospital",
  "policy_duration": null,
  "condition": "mental illness"
}
{
  "decision": "Approved",
  "amount": "Sum Insured as specified in the Poli

In [18]:
queries = ["Is Ayurvedic treatment covered under hospitalization benefits?", "Do I need to pay anything if I use cashless facility in a network hospital for surgery?", "Am I eligible for health check-up after one year of policy renewal?", "Is ambulance cost covered for transferring a patient from one hospital to another?"]
for query in queries:
  ask_a_question(query, "/content/BajajPolicies.pdf")

✅ Parsed Query: {
  "age": null,
  "gender": null,
  "procedure": "Ayurvedic treatment",
  "location": null,
  "policy_duration": null,
  "condition": "hospitalization benefits"
}
📄 Document loaded and split into 251 semantic chunks.
🔍 FAISS index built.
🔎 Retrieved 10 initial chunks from FAISS.
✨ Re-ranked chunks. Top 3 will be used as context.
{
  "decision": "Approved",
  "amount": "Up to Sum Insured",
  "justification": "As per Clause 8 on Page 11, Ayurvedic / Homeopathic Hospitalization Expenses are covered if the Insured is Hospitalized for not less than 24 hrs, in an Ayurvedic / Homeopathic Hospital which is a government Hospital or in any institute recognized by government and/or accredited by Quality Council of India/National Accreditation Board on Health, on the advice of a Medical Practitioner.",
  "clause_reference": "Clause 8 on Page 11"
}
✅ Parsed Query: {
  "age": null,
  "gender": null,
  "procedure": "surgery",
  "location": null,
  "policy_duration": null,
  "conditio

In [ ]:
ask_a_question("46M, knee surgery, Pune, 3-month policy", "/content/BAJHLIP23020V012223.pdf")

✅ Parsed Query: {
  "age": 46,
  "gender": "M",
  "procedure": "knee surgery",
  "location": "Pune",
  "policy_duration": "3-month",
  "condition": null
}
📄 Document loaded and split into 251 semantic chunks.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

✨ Re-ranked chunks. Top 3 will be used as context.
{
  "decision": "Approved",
  "amount": "Estimated payout amount of \u20b950,000 to \u20b970,000",
  "justification": "The policy covers medical expenses for knee surgery, and the estimated treatment cost falls within the limit of the policy.",
  "clause_reference": [
    "Page 43 Clause: 312 Revision/Removal of Knee cap ( Day Care Procedure)"
  ]
}


In [ ]:
ask_a_question("I am 20 years old which policies i can get for knee surgery", "/content/BAJHLIP23020V012223.pdf")

✅ Parsed Query: {
  "age": 20,
  "gender": null,
  "procedure": "knee surgery",
  "location": null,
  "policy_duration": null,
  "condition": null
}
📄 Document loaded and split into 251 semantic chunks.
🔍 FAISS index built.
🔎 Retrieved 10 initial chunks from FAISS.
✨ Re-ranked chunks. Top 3 will be used as context.
{
  "decision": "Rejected",
  "amount": "None",
  "justification": "The policy excludes cosmetic or plastic surgery (Code-Excl08) and surgery for body mass index (BMI) related issues (Code-Excl06). As the user is 20 years old, it's not clear if the knee surgery is elective or medically necessary. The policy does not provide coverage for elective procedures.",
  "clause_reference": "Page 25 - Code-Excl06 and Page 25 - Code-Excl08"
}


In [ ]:
ask_a_question("21M, knee surgery, Pune, 3-month policy", "/content/BAJHLIP23020V012223.pdf")


✅ Parsed Query: {
  "age": 21,
  "gender": "M",
  "procedure": "knee surgery",
  "location": "Pune",
  "policy_duration": 3,
  "condition": null
}
📄 Document loaded and split into 251 semantic chunks.
🔍 FAISS index built.
🔎 Retrieved 10 initial chunks from FAISS.
✨ Re-ranked chunks. Top 3 will be used as context.
{
  "decision": "Approved",
  "amount": "Rs. 3,00,000 (as per the policy terms)",
  "justification": "The policyholder has undergone knee surgery, which is a scheduled disease as per Annexure I- List of Day Care Procedures (page 43). Furthermore, the claim is submitted within 30 days of discharge from the hospital, as per Clause 45. Claims Procedure for International Cover- Reimbursement Claims and Pre-authorization Process for International Cover (page 37)",
  "clause_reference": [
    "Annexure I",
    "page 37"
  ]
}


In [ ]:
ask_a_question("My father, aged 58, is scheduled for a cataract surgery next week. Will our Bajaj policy cover the costs?", "/content/BAJHLIP23020V012223.pdf")

✅ Parsed Query: {
  "age": 58,
  "gender": null,
  "procedure": "cataract surgery",
  "location": null,
  "policy_duration": null,
  "condition": null
}
📄 Document loaded and split into 251 semantic chunks.
🔍 FAISS index built.
🔎 Retrieved 10 initial chunks from FAISS.
✨ Re-ranked chunks. Top 3 will be used as context.
{
  "decision": "Approved",
  "amount": "Estimated to be 80% of the total surgery costs",
  "justification": "The cataract surgery is an out-patient treatment covered under the Out-Patient Benefits for International Cover (Clause from Page 18, Section II). As the surgery is not related to dental treatment, dental implants, or orthodontics, it does not fall under any exclusion clause (Clause from Page 23, Section D, Item 1).",
  "clause_reference": "Clause from Page 18, Section II; Clause from Page 23, Section D, Item 1"
}


In [ ]:
ask_a_question("Is cataract operation included for a 58-year-old under this health policy from Bajaj?", "/content/BAJHLIP23020V012223.pdf")

✅ Parsed Query: {
  "age": 58,
  "gender": null,
  "procedure": "cataract operation",
  "location": null,
  "policy_duration": null,
  "condition": null
}
📄 Document loaded and split into 251 semantic chunks.
🔍 FAISS index built.
🔎 Retrieved 10 initial chunks from FAISS.
✨ Re-ranked chunks. Top 3 will be used as context.
{
  "decision": "Approved",
  "amount": "Not applicable (as it's a one-time surgery)",
  "justification": "The policy does not explicitly exclude cataract operations for individuals of the insured's age. The medical situation is specific to in-patient care and does not require an Act of Terrorism, War, or any other excluded circumstances. The Insured person is required to maintain accurate records and allow the company or its representatives to inspect them, which implies that the policyholder's compliance is a condition precedent to the obligation under the policy.",
  "clause_reference": "Based on Clause from Page 23: Section D) Exclusions\u2013 Specific Exclusions A

In [ ]:
ask_a_question("It's been 2 months since I bought the policy. Can I claim for gallbladder removal surgery now?", "/content/BAJHLIP23020V012223.pdf")

✅ Parsed Query: {
  "age": null,
  "gender": null,
  "procedure": "gallbladder removal surgery",
  "location": null,
  "policy_duration": 2,
  "condition": null
}
📄 Document loaded and split into 251 semantic chunks.
🔍 FAISS index built.
🔎 Retrieved 10 initial chunks from FAISS.
✨ Re-ranked chunks. Top 3 will be used as context.
{
  "decision": "Rejected",
  "amount": 0,
  "justification": "The procedure requires inpatient admission and gallbladder removal surgery cannot be performed on an out-patient basis. However, the policy has only been in effect for 2 months, and the claim timeline is not yet over.",
  "clause_reference": "UIN- BAJHLIP23020V012223 Global Health Care/ Policy Wordings/Page 11"
}


In [ ]:
ask_a_question("Will the insurance cover treatment for alcohol-related liver damage?", "/content/BAJHLIP23020V012223.pdf")

✅ Parsed Query: {
  "age": null,
  "gender": null,
  "procedure": "treatment for alcohol-related liver damage",
  "location": null,
  "policy_duration": null,
  "condition": "alcohol-related liver damage"
}
📄 Document loaded and split into 251 semantic chunks.
🔍 FAISS index built.
🔎 Retrieved 10 initial chunks from FAISS.
✨ Re-ranked chunks. Top 3 will be used as context.
{
  "decision": "Rejected",
  "amount": "0",
  "justification": "Treatment for alcohol-related liver damage is excluded under [Clause from Page 22] (Code -Excl12) which specifically excludes treatment for alcoholism, drug or substance abuse or any addictive condition and its consequences.",
  "clause_reference": "-Excl12"
}


In [ ]:
ask_a_question("Is a robotic knee replacement at Ruby Hall Clinic in Pune covered under this policy?", "/content/BAJHLIP23020V012223.pdf")

✅ Parsed Query: {
  "age": null,
  "gender": null,
  "procedure": "robotic knee replacement",
  "location": "Pune",
  "policy_duration": null,
  "condition": null
}
📄 Document loaded and split into 251 semantic chunks.
🔍 FAISS index built.
🔎 Retrieved 10 initial chunks from FAISS.
✨ Re-ranked chunks. Top 3 will be used as context.
{
  "decision": "Rejected",
  "amount": "N/A",
  "justification": "The policy specifically excludes treatment other than Allopathic treatment (Clause c, Page 16)",
  "clause_reference": "Page 16, Clause c"
}


In [ ]:

ask_a_question("What is the grace period for premium payment under the National Parivar Mediclaim Plus Policy?", "/content/BAJHLIP23020V012223.pdf")


✅ Parsed Query: {
  "age": null,
  "gender": null,
  "procedure": null,
  "location": null,
  "policy_duration": null,
  "condition": "National Parivar Mediclaim Plus Policy"
}
📄 Document loaded and split into 251 semantic chunks.
🔍 FAISS index built.
🔎 Retrieved 10 initial chunks from FAISS.
✨ Re-ranked chunks. Top 3 will be used as context.
JSON parsing error: Expecting value: line 3 column 11 (char 36)
{
  "error": "Could not parse output",
  "raw": "Here is the response in JSON format:\n\n{\n\"decision\": \"Approved\",\n\"amount\": None,\n\"justification\": \"The policy provides a 15-day grace period to pay the instalment premium due for the policy.\",\n\"clause_reference\": \"Page 31, Clause 13(i)\"\n}"
}
